In [436]:
from flask import Flask, render_template, request, redirect, url_for, session, flash
from pdf2image import convert_from_path, pdfinfo_from_path
from werkzeug.utils import secure_filename
from cryptography.fernet import Fernet
from flask_session import Session
from flask_mysqldb import MySQL
from flask import jsonify
import MySQLdb.cursors
from PIL import Image
import pandas as pd
import pytesseract
import threading
import requests
import shutil
import base64
import yake
import json
import sys
import re
import os

In [437]:
df = pd.read_csv('Q&A.csv',header=0)

kw_extractor = yake.KeywordExtractor()
language = "en"
max_ngram_size = 2
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)

UPLOAD_FOLDER = 'books'
ALLOWED_EXTENSIONS = {'pdf'}

app = Flask(__name__)
app.secret_key = os.urandom(24)
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'godsvoice'
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['converted_books'] = 'converted_books'
mysql = MySQL(app)
Session(app)
key = Fernet.generate_key()
f = Fernet(key)

In [438]:
def pdf2text(file_name):
    PDF_file=os.path.join(app.config['UPLOAD_FOLDER'], file_name)
    outfile = "output_"+str(PDF_file.split(".")[0])+".txt"
    image_counter = 1
    info = pdfinfo_from_path(PDF_file, userpw=None, poppler_path=None)
    maxPages =info["Pages"]
    page_interval=2
    for i in range(1, maxPages+1, page_interval) : 
        pages=convert_from_path(PDF_file, dpi=700, first_page=i, last_page = min(i+page_interval-1,maxPages))
        for page in pages:
            filename = str(PDF_file.split(".")[0])+"_page_"+str(image_counter)+".jpg"
            page.save(filename, 'JPEG')
            image_counter = image_counter + 1
            del page
        del pages
    filelimit = image_counter-1
    f = open(outfile, "w")
    for i in range(1, filelimit + 1):
        filename = str(PDF_file.split(".")[0])+"_page_"+str(i)+".jpg"
        text = str(((pytesseract.image_to_string(Image.open(filename),config=r'--tessdata-dir "C:\Program Files\Tesseract-OCR\tessdata"'))))
        text = text.replace('-\n', '') 
        text = text.replace('', '') 
        f.write(text)
        os.remove(filename)
    f.close()
    shutil.move(PDF_file, app.config['converted_books'])

    
class ReCaptcha(object):

    VERIFY_URL = "https://www.google.com/recaptcha/api/siteverify"
    site_key = '6Ldvg90bAAAAAJxlCJLQs9K2XJsak2Q-dZgkVBWH'
    secret_key = '6Ldvg90bAAAAAJFJkfsIGNoBni5-eBoEKkISV_S2'
    is_enabled = True
    def verify(self, response=None, remote_ip=None):
        if self.is_enabled:
            data = {
                "secret": self.secret_key,
                "response": response or request.form.get('g-recaptcha-response'),
                "remoteip": remote_ip or request.environ.get('REMOTE_ADDR')
            }

            r = requests.get(self.VERIFY_URL, params=data)
            return r.json()["success"] if r.status_code == 200 else False
        return True

In [439]:
recaptcha = ReCaptcha()

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/admin', methods =['GET', 'POST'])
def admin():
    msg = ''
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form:
        if recaptcha.verify():
            username = request.form['username']
            password = request.form['password']
            pwd=base64.b64encode(password.encode("utf-8"))
            cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
            cursor.execute('SELECT * FROM accounts WHERE username = % s AND password = % s AND is_deleted=0', (username, pwd, ))
            account = cursor.fetchone()
            if account:
                session['admin'] = account['is_admin']
                session['superadmin'] = account['is_superadmin']
                if session['admin']==1:
                    session['loggedin'] = True
                    session['id'] = account['id']
                    session['username'] = account['username']
                    session['admin'] = account['is_admin']
                    if session['superadmin']==1:
                        return render_template('admin.html', msg = msg, superadmin=session['superadmin'])
                    return render_template('admin.html', msg = msg, superadmin=session['superadmin'])
                else:
                    session['loggedin'] = False
                    msg = 'Your account is not verified by admin !'
                    return render_template('login.html', msg = msg)
            else:
                session['loggedin'] = False
                msg = 'Incorrect username / password !'
        else:
            session['loggedin'] = False
            msg = 'Please verify captcha'
    if session.get('loggedin')==True:
        if session['superadmin']==1:
            if session['msg']:
                msg=session['msg']
                session['msg']=''
            return render_template('admin.html', msg = msg, superadmin=session['superadmin'])
        return render_template('admin.html', msg = msg, superadmin=session['superadmin'])
    return render_template('login.html', msg = msg)

@app.route('/logout')
@app.route('/logout', methods =['GET', 'POST'])
def logout():
    session['loggedin'] = False
    session.pop('id', None)
    session.pop('username', None)
    return redirect(url_for('admin'))

@app.route('/register', methods =['GET', 'POST'])
def register():
    msg = ''
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form and 'email' in request.form :
        if recaptcha.verify():
            username = request.form['username']
            password = request.form['password']
            email = request.form['email']
            cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
            cursor.execute('SELECT * FROM accounts WHERE username = % s AND is_deleted=0', (username, ))
            account = cursor.fetchone()
            if not username or not password or not email:
                msg = 'Please fill out the form !'
            elif account:
                msg = 'Account already exists !'
            elif not re.match(r'[^@]+@[^@]+\.[^@]+', email):
                msg = 'Invalid email address !'
            elif not re.match(r'[A-Za-z0-9]+', username):
                msg = 'Username must contain only characters and numbers !'
            else:
                pwd=base64.b64encode(password.encode("utf-8"))
                cursor.execute('INSERT INTO accounts VALUES (NULL, % s, % s, % s, 0, 0, 0)', (username, pwd, email, ))
                mysql.connection.commit()
                msg = 'You have successfully registered !'
                return redirect(url_for('admin'))
        else:
            session['loggedin'] = False
            msg = 'Please verify captcha'
    elif request.method == 'POST':
        msg = 'Please fill out the form !'
    elif session.get('loggedin')==True:
        return redirect(url_for('admin'))
    return render_template('register.html', msg = msg)

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    session['msg']=''
    if request.method == 'POST' and 'Book' in request.form and 'Author' in request.form:
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        Book_name = request.form['Book']
        Author = request.form['Author'].split(',')
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute('SELECT * FROM books WHERE book_name = % s', (Book_name, ))
        book = cursor.fetchone()
        if book:
            session['msg'] = 'Book already exists !'
        elif file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            cursor.execute('SELECT * FROM books WHERE filename = % s', (filename, ))
            file_name= cursor.fetchone()
            if file_name:
                session['msg'] = 'Book with same file name already exists !'
                return redirect(url_for('admin'))
            cursor.execute('INSERT INTO books VALUES (NULL, % s, %s )', (Book_name, filename,))
            for i in Author:
                cursor.execute('SELECT * FROM authors WHERE author_name = % s', (i, ))
                authors = cursor.fetchone()
                if not authors:
                    cursor.execute('INSERT INTO authors VALUES (NULL, % s)', (i.strip(), ))
                cursor.execute('SELECT * FROM authors WHERE author_name = % s', (i.strip(), ))
                authors = cursor.fetchone()
                cursor.execute('SELECT * FROM books WHERE book_name = % s', (Book_name, ))
                book = cursor.fetchone()
                cursor.execute('INSERT INTO book_author_rel  VALUES (%s, %s )', (book['b_id'],authors['a_id'],))                
            mysql.connection.commit()    
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))                    
            t1 = threading.Thread(target=pdf2text, args=(filename,))
            t1.start()
            session['msg'] = 'Book Uploaded!'
            return redirect(url_for('admin'))
    return redirect(url_for('admin'))

@app.route('/users')
def users():
    if session.get('loggedin')==True:
        if session['superadmin']==1:
            cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
            cursor.execute('SELECT * FROM accounts WHERE is_deleted=0')
            account = cursor.fetchall()
            return render_template('users.html', data=account, current=session['username'])
        return redirect(url_for('admin'))
    return redirect(url_for('admin'))

@app.route('/update/<user_id>/<admin_type>/<value>', methods=['GET', 'POST'])
def update(user_id,admin_type,value):
    if session.get('loggedin')==True:
        if session['superadmin']==1:
            if admin_type=='is_superadmin':
                querry='UPDATE accounts SET '+admin_type+'='+value+', is_admin=1 WHERE id='+user_id
            else:
                querry='UPDATE accounts SET '+admin_type+'='+value+', is_superadmin=0 WHERE id='+user_id
            cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
            cursor.execute(querry)
            mysql.connection.commit()
            return redirect(url_for('users'))
    return redirect(url_for('admin'))

@app.route('/delete/<user_id>', methods=['GET', 'POST'])
def delete(user_id):
    if session.get('loggedin')==True:
        if session['superadmin']==1:
            querry='UPDATE accounts SET is_admin=0, is_superadmin=0, is_deleted=1 WHERE id='+user_id
            cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
            cursor.execute(querry)
            mysql.connection.commit()
            return redirect(url_for('users'))
    return redirect(url_for('admin'))

In [440]:
@app.route('/')
def homes():
    return render_template('/index.html')

@app.route('/',methods = ['POST', 'GET'])
def home():
    return render_template('/index.html')

@app.route('/books')
def books():
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute('SELECT * FROM books')
    account = cursor.fetchall()
    return render_template('/book.html', data=account)

@app.route('/author')
def author():
    cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cursor.execute('SELECT * FROM authors')
    account = cursor.fetchall()
    return render_template('/author.html', data=account)

@app.route('/search',methods = ['POST', 'GET'])
def search():
    size=0
    keywords_list=[]
    if request.method == 'POST':
        question = request.form['question']
        keywords = custom_kw_extractor.extract_keywords(question)
        for i in keywords:
            keywords_list.append(i[0])
            size=size+1
        
        if keywords_list=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('result',keywords_list = json.dumps(keywords_list),size=size))
    else:
        question = request.form['question']
        keywords = custom_kw_extractor.extract_keywords(question)
        for i in keywords:
            keywords_list.append(i[0])
            size=size+1
        if keywords_list=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('result',keywords_list = json.dumps(keywords_list),size=size))

@app.route('/result/<keywords_list>/<size>')
def result(keywords_list,size):
    questions_list=[]
    size=int(size)
    lst = json.loads(keywords_list)
    if size>1:
        for j in lst:
            for i in range(len(df['questions'])):
                if j in df['questions'][i]:
                    if not df['questions'][i] in questions_list:
                        questions_list.append(df['questions'][i])
    else:
        for i in range(len(df['questions'])):
            if keywords_list in df['questions'][i]:
                if not df['questions'][i] in questions_list:
                    questions_list.append(df['questions'][i])
    return render_template('/result.html',value = questions_list)

@app.route('/answer',methods = ['POST', 'GET'])
def answer():
    if request.method == 'POST':
        question = request.form['question']
        answer=''
        for i in range(len(df['questions'])):
            if df['questions'][i] == question:
                answer=df['answers'][i]
        
        if answer=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('success',answer = answer))
    else:
        question = request.form['question']
        answer=''
        for i in range(len(df['questions'])):
            if df['questions'][i] == question:
                answer=df['answers'][i]
        if answer=='':
            return redirect(url_for('home'))
        else:
            return redirect(url_for('success',answer = answer))

@app.route('/success/<answer>')
def success(answer):
    return render_template('answer.html', answer = answer)

In [441]:
if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.32:5000/ (Press CTRL+C to quit)
192.168.1.32 - - [11/Aug/2021 13:51:51] "GET /author HTTP/1.1" 200 -
192.168.1.32 - - [11/Aug/2021 13:51:51] "GET /static/css/new.css HTTP/1.1" 200 -
192.168.1.32 - - [11/Aug/2021 13:51:51] "GET /static/css/new_1.css HTTP/1.1" 200 -
192.168.1.32 - - [11/Aug/2021 13:51:51] "GET /static/images/Searchs_130.png HTTP/1.1" 200 -
